---

## 1. `requires_grad`：是否追踪梯度（Autograd 开关）

在 PyTorch 中，`requires_grad` 是 Tensor 的核心属性，用来标记**是否需要对该张量计算梯度**。

### 1.1 开启梯度追踪：`requires_grad=True`

- 当 `requires_grad=True` 时，PyTorch 会在**前向传播**中记录该张量参与的运算，构建**计算图（computation graph）**。
- 之后调用 `backward()` 时，能够沿计算图自动求导，得到梯度。

示例：

```python
import torch

a = torch.randn((), requires_grad=True)
d = a * a + 1
d.backward()
print(a.grad)  # dd/da
```

### 1.2 关闭梯度追踪：`requires_grad=False`（默认）

- 不记录计算图，也不计算梯度，节省内存与计算。
- 常用于：输入数据、固定参数、推理阶段等。

---

## 2. `.backward()` / `.grad` / `zero_()`：Autograd 三个基本动作

### 2.1 `x.grad.zero_()`：清空梯度（防止梯度累加）

PyTorch 默认会**累加**梯度：如果不清零，下一次 `backward()` 得到的梯度会加到旧梯度上。

```python
x.grad.zero_()
```

训练中更常见的是：

```python
optimizer.zero_grad()
```

### 2.2 `y.backward()`：反向传播计算梯度

- `backward()` 会沿计算图反向传播，计算所有 `requires_grad=True` 的叶子张量的梯度。
- 结果一般存放在对应张量的 `.grad` 中。

### 2.3 `x.grad`：查看梯度

- `.grad` 的形状通常与 `x` 的形状一致（“梯度形状 = 参数形状”）。

---

## 3. “非标量怎么求导”：`sum()`（或 `mean()`）把输出变成标量

PyTorch 中最常用的方式是：**把向量/矩阵输出先聚合成标量**再反传。

### 3.1 `sum()` 求导示例：`y = x.sum()`

```python
import torch

x = torch.arange(4.0, requires_grad=True)  # tensor([0.,1.,2.,3.])
y = x.sum()                                # 标量
y.backward()
print(x.grad)  # tensor([1., 1., 1., 1.])
```

### 3.2 “求导的求和技巧”：`y = x*x` 但用 `y.sum().backward()`

```python
x = torch.arange(4.0, requires_grad=True)
x.grad.zero_()

y = x * x                 # tensor([0.,1.,4.,9.])  (非标量)
y.sum().backward()        # 关键：先 sum 成标量再 backward
print(x.grad)             # tensor([0., 2., 4., 6.])  -> 2x
```

---

## 4. `detach()`：把某些计算“移到计算图之外”（切断梯度流）

### 4.1 核心含义

`u = y.detach()` 的意思：

- `u` **拿到 `y` 当前的数值**；
- 但 `u` **不再连接到产生 `y` 的计算图**；
- 因此从 `u` 出发的后续计算，梯度不会回传到 `y` 的来源（链式法则在此断开）。

### 4.2 为什么用了 `detach()` 结果会变？

对比两种情况（非常常见的 D2L 讲法）：

#### 情况 A：使用 `detach()`

```python
x = torch.arange(4.0, requires_grad=True)
x.grad.zero_()

y = x * x          # y 与 x 有计算图关系
u = y.detach()     # u 与 y 断开：u 被当作常量（对 x 不再可导）
z = u * x          # z = (常量u) * x

z.sum().backward()
print("x.grad =", x.grad)
print("u      =", u)
```

- 因为 `u` 被视作常量，`z = u*x` 对 `x` 的导数就是 `u`（逐元素）。

#### 情况 B：不使用 `detach()`

```python
x = torch.arange(4.0, requires_grad=True)
x.grad.zero_()

y = x * x
z = y * x          # z = x^2 * x = x^3

z.sum().backward()
print(x.grad)      # 逐元素应为 3*x^2
```

### 4.3 典型用途（D2L 常见语境）

- 固定某一部分计算/参数，不希望梯度更新它；
- 只想用某个中间结果的“当前值”，但不想让梯度穿过该分支。

---

## 5. `sum(axis/dim=...)` 的维度变化 + `keepdims/keepdim` 的作用

这部分是 D2L 里非常核心的“维度直觉”。

### 5.1 二维张量：`(5,4)` 的例子

假设 `A.shape = (5,4)`：

- `A.sum(axis=0)`：对 **列** 求和（沿行方向累加），结果形状通常是 `(4,)`
- `A.sum(axis=1)`：对 **行** 求和（沿列方向累加），结果形状通常是 `(5,)`

在 PyTorch 中写法通常是：

- `A.sum(dim=0)` 或 `A.sum(dim=1)`

### 5.2 `keepdims/keepdim`：求和后是否保留被压缩的维度

- 默认 `keepdim=False`：被求和的那一维会**消失**。
- 若 `keepdim=True`：那一维会保留为长度为 1 的维度，方便后续广播对齐。

例：`A.shape = (5,4)`

- `A.sum(dim=1)` -> shape `(5,)`
- `A.sum(dim=1, keepdim=True)` -> shape `(5,1)`

---

## 6. 广播（Broadcasting）与“行归一化”：为什么常常需要 `keepdim=True`

### 6.1 正确的逐行归一化写法

目标：每一行除以该行的和，使每行元素和为 1。

```python
import torch

A = torch.tensor([[1., 3.],
                  [2., 4.]])

sum_A = A.sum(dim=1, keepdim=True)   # shape (2,1), 值 [[4],[6]]
normalized = A / sum_A

print(sum_A)
print(normalized)
```

输出应类似：

- `sum_A = [[4.],[6.]]`
- `normalized = [[0.25, 0.75], [0.3333, 0.6667]]`（每行和为 1）

### 6.2 不用 `keepdim=True` 可能产生“错位广播”

```python
sum_A_bad = A.sum(dim=1)   # shape (2,) -> [4,6]
normalized_bad = A / sum_A_bad
print(normalized_bad)
```

这种写法会触发广播，但广播对齐方式可能导致结果**不是逐行除以本行和**（容易出现错位/逻辑错误），因此做“按行/按列归一化”时，D2L 强烈建议用 `keepdim=True` 保持维度对齐。

---

## 7. 三维张量的 `sum`：`shape = (2,5,4)` 时 `axis/dim` 怎么理解

设 `a = torch.ones((2,5,4))`：

```python
import torch
a = torch.ones((2,5,4))
print(a.shape)                 # torch.Size([2, 5, 4])

b = a.sum(dim=1)
print(b.shape)                 # torch.Size([2, 4])
print(b)                       # 每个元素为 5（因为 dim=1 那一维长度是 5）
```

- `sum(dim=1)`：把中间那一维（长度 5）求和掉，所以从 `(2,5,4)` 变成 `(2,4)`
- 若 `keepdim=True`：
  - `a.sum(dim=1, keepdim=True)` 形状为 `(2,1,4)`
- 多维一起求和（概念上与 D2L 一致）：
  - `a.sum(dim=(1,2), keepdim=True)` 形状为 `(2,1,1)`

---

## 8. `cumsum`：沿某个轴计算“累积和”（cumulative sum）

`cumsum` 会从起点开始，把当前元素与之前元素不断累加。

### 8.1 `axis/dim=0` 的含义

- `dim=0`：沿“行”方向累积（对每一列分别做从上到下的累加）。

示例（5×4）：

```python
import torch
A = torch.arange(20.0).reshape(5,4)
print(A)

C = A.cumsum(dim=0)
print(C)
```

直观理解（按列累加）：

- 第 1 行：不变
- 第 2 行：第1行 + 第2行
- 第 3 行：前两行累计 + 第3行
- ...
- 最后一行：就是该列的总累积结果

---

## 9. 行向量 vs 列向量：以及 `torch.mv` 的“向量隐形处理”

### 9.1 行/列向量概念

- 行向量：形状 `1×n`
- 列向量：形状 `n×1`
- 一维张量 `x.shape == (n,)` 在不同算子里可能有约定处理。

### 9.2 `torch.mv(A, x)`：矩阵 × 向量

- `torch.mv` 专门用于 **(m×n) 矩阵** 乘 **(n,) 向量**。
- 你看到 `x.shape=(n,)` 虽是 1D，但在 `mv` 语义里会按“向量”规则处理，使维度匹配 `A @ x`。

示例：

```python
import torch

A = torch.randn(5,4)
x = torch.randn(4)          # shape (4,)
b = torch.mv(A, x)          # shape (5,)
print(b.shape)
```

---

## 10. `torch.mm(A, B)`：矩阵 × 矩阵（2D × 2D）

`torch.mm` 是 **矩阵乘法**（matrix-matrix multiply）：

- 输入必须都是 2D
- 维度规则：`A(m×n) @ B(n×p) -> C(m×p)`

示例：

```python
import torch

A = torch.arange(20.0).reshape(5,4)   # 5×4
B = torch.ones(4,3)                   # 4×3
C = torch.mm(A, B)                    # 5×3
print(C.shape)
print(C)
```

如果 `B` 每列都是全 1，则 `C` 的每个元素是“对应行元素之和”（并在每列重复）。

---

## 11. 建议练习（与截图一致）：把 `x.sum()` 升级为 `(x*x).sum()`

目的：亲手验证梯度从 `1` 变成 `2x`。

```python
import torch

x = torch.arange(4.0, requires_grad=True)
if x.grad is not None:
    x.grad.zero_()

y = (x * x).sum()
y.backward()
print(x)
print(x.grad)   # 应为 2*x
```

---

## 12. 本章（对应 D2L 语境）你需要真正“吃透”的东西（不靠背公式）

- 会用：`requires_grad=True`
- 会用：`backward()`
- 记得：梯度会累加 -> 每轮先 `zero_grad()` / `grad.zero_()`
- 非标量输出：常用 `sum()` / `mean()` 变标量再反传
- 需要“断梯度”：`detach()`
- 维度/广播：`sum(dim=..., keepdim=True)` 是很多归一化/广播正确性的关键
- 线代接口区分：`mv`（矩阵×向量） vs `mm`（矩阵×矩阵）

```

```
